In [1]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,GlobalMaxPool1D, Bidirectional 
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from numpy import random
from random import choices
import os
from sklearn.utils import shuffle
from sklearn import metrics
import pickle
from scipy.io.wavfile import read,write
import pickle

from datetime import datetime
from packaging import version

tf.keras.backend.set_floatx('float32')

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.4.1


In [4]:
os.chdir('/content/drive/My Drive/')
os.listdir()

['depthwise_conv2d.py',
 'Colab Notebooks',
 'logs',
 'Pickles',
 'Zips',
 'Data',
 'Help_Models',
 'Bachao_Models',
 'model.py',
 'data_preprocess.py',
 'utils.py',
 'CRoam Model Training  Report.pdf',
 'speaker_ver1_model.h5',
 'speaker_ver1_model.pb',
 'speaker_ver1a_model.h5',
 'speaker_ver1b_model.h5',
 'speaker_ver1a_model.pb',
 'graph_def_for_reference.pb.ascii',
 'speaker_ver1b_model.pb',
 'new_mix_help_train_x.pickle',
 'new_mix_help_train_y.pickle',
 'total_bachao_x_new.pickle',
 'total_bachao_y_new.pickle']

In [5]:
from keras.models import load_model
model = load_model('../content/drive/My Drive/Bachao_Models/HDF5/newbachao.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)
# [<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>]

OSError: ignored

In [7]:
#from tensorflow.python.keras import backend as K
import tensorflow.python.keras.backend as K

#from tensorflow.compat.v1.keras import backend as K
#K.set_session
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])


TypeError: ignored

In [ ]:
from keras import backend as K

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
from keras.models import load_model

#loaded_model=load_model('cardogcpunogray.hdf5')
model = load_model('/content/drive/My Drive/Bachao_Models/HDF5/newbachao.h5')


MODEL_NAME="cardogcpunogray"
output_node_name="lastlayer/Softmax"
input_node_names=["conv2d_1_input"]
saver=tf.train.Saver()

tf.train.write_graph(K.get_session().graph_def, 'out', MODEL_NAME + '_graph.pbtxt')


tf.train.Saver().save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')


freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, False, 'out/' + MODEL_NAME + '.chkp', output_node_name, "save/restore_all", "save/Const:0", 'out1/frozen_' + MODEL_NAME + '.pb', True, "")

input_graph_def = tf.GraphDef()
with tf.gfile.Open('out1/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

with tf.gfile.FastGFile('out1/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

print("graph saved!")



"""
 from tensorflow.tools.graph_transforms import TransformGraph
    transforms = ["quantize_weights", "quantize_nodes"]
    transformed_graph_def = TransformGraph(sess.graph.as_graph_def(), [], pred_node_names, transforms)
    constant_graph = graph_util.convert_variables_to_constants(sess, transformed_graph_def, pred_node_names)
"""

AttributeError: ignored